In [ ]:
!pip install natsort

In [ ]:

import os
import cv2
import numpy as np
from PIL import Image,ImageOps
from zipfile import ZipFile
from natsort import natsorted


def crop_img(img, scale=1.0):
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    center_x, center_y = img.shape[1] / 2, img.shape[0] / 2
    width_scaled, height_scaled = img.shape[1] * scale, img.shape[0] * scale
    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
    img_cropped = img[int(top_y):int(bottom_y), int(left_x):int(right_x)]
    im_pil = Image.fromarray(img_cropped)
    im_pil = ImageOps.grayscale(im_pil)
    return im_pil



def convert_one_channel(image):

    if len(image.shape)>2:
        image=image[:,:,0]
        return image
    else:
        return image
    
def images(resize_shape, path):
   
    dirs=natsorted(os.listdir(path))
    images=image=Image.open(path+dirs[0]).convert('RGB')
    images=(images.resize((resize_shape),Image.ANTIALIAS))
    images=convert_one_channel(np.asarray(images))
    for i in range (1,len(dirs)):
        image=Image.open(path+dirs[i]).convert('RGB')
        image=image.resize((resize_shape),Image.ANTIALIAS)
        image=convert_one_channel(np.asarray(image))
        images=np.concatenate((images,image))
        
    images=np.reshape(images,(len(dirs),resize_shape[0],resize_shape[1],1))
    
    return images

path="/kaggle/input/croppedcavities/croppedX/"

X=images((512,512),path)

In [3]:
import os
import sys
import numpy as np
import cv2
from PIL import Image
from zipfile import ZipFile
from natsort import natsorted

def convert_one_channel(img):
    if len(img.shape)>2:
        img=img[:,:,0]
        return img
    else:
        return img


def masks(path):
    path="/kaggle/input/croppedcavities/croppedYCavities/"
    dirs=natsorted(os.listdir(path))
    masks=image=Image.open(path+dirs[0])
    masks=convert_one_channel(np.asarray(masks))
    for i in range (1,len(dirs)):
        image=Image.open(path+dirs[i])
        image=convert_one_channel(np.asarray(image))
        masks=np.concatenate((masks,image))
    masks=np.reshape(masks,(len(dirs),512,512,1))
    return masks
    

Y=masks(path="/kaggle/input/croppedcavities/croppedYCavities/")


In [ ]:
import math
src=X[0,:,:,0]
mid = 0.5
mean = np.mean(src)
gamma = math.log(mid*255)/math.log(mean)
        
X2= np.power(src, gamma).clip(0,255).astype(np.uint8)
X2 = cv2.cvtColor(X2,cv2.COLOR_GRAY2RGB)

for i in range(len(X)):
    if i!=0:
        src=X[i,:,:,0]
        mid = 0.5
        mean = np.mean(src)
        gamma = math.log(mid*255)/math.log(mean)
        dst = np.power(src, gamma).clip(0,255).astype(np.uint8)
        dst = cv2.cvtColor(dst,cv2.COLOR_GRAY2RGB) #for 3 channels
        X2=np.concatenate((X2,dst))
X2=np.reshape(X2,(390,512,512,3))
print(X2.shape)
X=X2


In [ ]:
X=np.float32(X/255)
Y=np.float32(Y/255)

In [ ]:
x_train=X[:97,:,:,:]
y_train=Y[:97,:,:,:]
x_test=X[97:,:,:,:]
y_test=Y[97:,:,:,:]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(Y[100,:,:,0], cmap="gray")

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X[17,:,:,0], cmap="gray")

In [ ]:
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

In [10]:
import cv2

import albumentations as A


aug = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.35,p=0.5),
    A.ShiftScaleRotate (shift_limit=0.10, scale_limit=0.10, rotate_limit=15,border_mode=cv2.BORDER_CONSTANT, p=1),
    A.Rotate(15,p=0.5),
    A.HorizontalFlip(p=0.45),
    A.Sharpen(p=0.2),
])


x_train1=np.copy(x_train)
y_train1=np.copy(y_train)
count=0
while(count<2):
  x_aug2=np.copy(x_train1)
  y_aug2=np.copy(y_train1)
  for i in range(len(x_train1)):
    augmented=aug(image=x_train1[i,:,:,:],mask=y_train1[i,:,:,:])
    x_aug2[i,:,:,:]= augmented['image']
    y_aug2[i,:,:,:]= augmented['mask']
  x_train=np.concatenate((x_train,x_aug2))
  y_train=np.concatenate((y_train,y_aug2))
  if count == 9:
    break
  count += 1
x_train=np.concatenate((x_train,x_train1))
y_train=np.concatenate((y_train,y_train1))
print(len(x_train))

376


In [ ]:
%env SM_FRAMEWORK=tf.keras

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
import segmentation_models as sm

In [ ]:
#source: bonlime (Github)
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Dropout
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.python.keras.utils.layer_utils import get_source_inputs
from tensorflow.python.keras.utils.data_utils import get_file
from tensorflow.keras import backend as K
from tensorflow.keras.applications.imagenet_utils import preprocess_input

WEIGHTS_PATH_X = "https://github.com/bonlime/keras-deeplab-v3-plus/releases/download/1.1/deeplabv3_xception_tf_dim_ordering_tf_kernels.h5"
WEIGHTS_PATH_MOBILE = "https://github.com/bonlime/keras-deeplab-v3-plus/releases/download/1.1/deeplabv3_mobilenetv2_tf_dim_ordering_tf_kernels.h5"
WEIGHTS_PATH_X_CS = "https://github.com/bonlime/keras-deeplab-v3-plus/releases/download/1.2/deeplabv3_xception_tf_dim_ordering_tf_kernels_cityscapes.h5"
WEIGHTS_PATH_MOBILE_CS = "https://github.com/bonlime/keras-deeplab-v3-plus/releases/download/1.2/deeplabv3_mobilenetv2_tf_dim_ordering_tf_kernels_cityscapes.h5"


def SepConv_BN(x, filters, prefix, stride=1, kernel_size=3, rate=1, depth_activation=False, epsilon=1e-3):
    """ SepConv with BN between depthwise & pointwise. Optionally add activation after BN
        Implements right "same" padding for even kernel sizes
        Args:
            x: input tensor
            filters: num of filters in pointwise convolution
            prefix: prefix before name
            stride: stride at depthwise conv
            kernel_size: kernel size for depthwise convolution
            rate: atrous rate for depthwise convolution
            depth_activation: flag to use activation between depthwise & poinwise convs
            epsilon: epsilon to use in BN layer
    """

    if stride == 1:
        depth_padding = 'same'
    else:
        kernel_size_effective = kernel_size + (kernel_size - 1) * (rate - 1)
        pad_total = kernel_size_effective - 1
        pad_beg = pad_total // 2
        pad_end = pad_total - pad_beg
        x = ZeroPadding2D((pad_beg, pad_end))(x)
        depth_padding = 'valid'

    if not depth_activation:
        x = Activation(tf.nn.relu)(x)
    x = DepthwiseConv2D((kernel_size, kernel_size), strides=(stride, stride), dilation_rate=(rate, rate),
                        padding=depth_padding, use_bias=False, name=prefix + '_depthwise')(x)
    x = BatchNormalization(name=prefix + '_depthwise_BN', epsilon=epsilon)(x)
    if depth_activation:
        x = Activation(tf.nn.relu)(x)
    x = Conv2D(filters, (1, 1), padding='same',
               use_bias=False, name=prefix + '_pointwise')(x)
    x = BatchNormalization(name=prefix + '_pointwise_BN', epsilon=epsilon)(x)
    if depth_activation:
        x = Activation(tf.nn.relu)(x)

    return x


def _conv2d_same(x, filters, prefix, stride=1, kernel_size=3, rate=1):
    """Implements right 'same' padding for even kernel sizes
        Without this there is a 1 pixel drift when stride = 2
        Args:
            x: input tensor
            filters: num of filters in pointwise convolution
            prefix: prefix before name
            stride: stride at depthwise conv
            kernel_size: kernel size for depthwise convolution
            rate: atrous rate for depthwise convolution
    """
    if stride == 1:
        return Conv2D(filters,
                      (kernel_size, kernel_size),
                      strides=(stride, stride),
                      padding='same', use_bias=False,
                      dilation_rate=(rate, rate),
                      name=prefix)(x)
    else:
        kernel_size_effective = kernel_size + (kernel_size - 1) * (rate - 1)
        pad_total = kernel_size_effective - 1
        pad_beg = pad_total // 2
        pad_end = pad_total - pad_beg
        x = ZeroPadding2D((pad_beg, pad_end))(x)
        return Conv2D(filters,
                      (kernel_size, kernel_size),
                      strides=(stride, stride),
                      padding='valid', use_bias=False,
                      dilation_rate=(rate, rate),
                      name=prefix)(x)


def _xception_block(inputs, depth_list, prefix, skip_connection_type, stride,
                    rate=1, depth_activation=False, return_skip=False):
    """ Basic building block of modified Xception network
        Args:
            inputs: input tensor
            depth_list: number of filters in each SepConv layer. len(depth_list) == 3
            prefix: prefix before name
            skip_connection_type: one of {'conv','sum','none'}
            stride: stride at last depthwise conv
            rate: atrous rate for depthwise convolution
            depth_activation: flag to use activation between depthwise & pointwise convs
            return_skip: flag to return additional tensor after 2 SepConvs for decoder
            """
    residual = inputs
    for i in range(3):
        residual = SepConv_BN(residual,
                              depth_list[i],
                              prefix + '_separable_conv{}'.format(i + 1),
                              stride=stride if i == 2 else 1,
                              rate=rate,
                              depth_activation=depth_activation)
        if i == 1:
            skip = residual
    if skip_connection_type == 'conv':
        shortcut = _conv2d_same(inputs, depth_list[-1], prefix + '_shortcut',
                                kernel_size=1,
                                stride=stride)
        shortcut = BatchNormalization(name=prefix + '_shortcut_BN')(shortcut)
        outputs = layers.add([residual, shortcut])
    elif skip_connection_type == 'sum':
        outputs = layers.add([residual, inputs])
    elif skip_connection_type == 'none':
        outputs = residual
    if return_skip:
        return outputs, skip
    else:
        return outputs


def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def _inverted_res_block(inputs, expansion, stride, alpha, filters, block_id, skip_connection, rate=1):
    in_channels = inputs.shape[-1]
    pointwise_conv_filters = int(filters * alpha)
    pointwise_filters = _make_divisible(pointwise_conv_filters, 8)
    x = inputs
    prefix = 'expanded_conv_{}_'.format(block_id)
    if block_id:
        # Expand

        x = Conv2D(expansion * in_channels, kernel_size=1, padding='same',
                   use_bias=False, activation=None,
                   name=prefix + 'expand')(x)
        x = BatchNormalization(epsilon=1e-3, momentum=0.999,
                               name=prefix + 'expand_BN')(x)
        x = Activation(tf.nn.relu6, name=prefix + 'expand_relu')(x)
    else:
        prefix = 'expanded_conv_'
    # Depthwise
    x = DepthwiseConv2D(kernel_size=3, strides=stride, activation=None,
                        use_bias=False, padding='same', dilation_rate=(rate, rate),
                        name=prefix + 'depthwise')(x)
    x = BatchNormalization(epsilon=1e-3, momentum=0.999,
                           name=prefix + 'depthwise_BN')(x)

    x = Activation(tf.nn.relu6, name=prefix + 'depthwise_relu')(x)

    # Project
    x = Conv2D(pointwise_filters,
               kernel_size=1, padding='same', use_bias=False, activation=None,
               name=prefix + 'project')(x)
    x = BatchNormalization(epsilon=1e-3, momentum=0.999,
                           name=prefix + 'project_BN')(x)

    if skip_connection:
        return Add(name=prefix + 'add')([inputs, x])

    # if in_channels == pointwise_filters and stride == 1:
    #    return Add(name='res_connect_' + str(block_id))([inputs, x])

    return x


def Deeplabv3(weights='pascal_voc', input_tensor=None, input_shape=(512, 512, 3), classes=21, backbone='mobilenetv2',
              OS=16, alpha=1., activation=None):
    """ Instantiates the Deeplabv3+ architecture
    Optionally loads weights pre-trained
    on PASCAL VOC or Cityscapes. This model is available for TensorFlow only.
    # Arguments
        weights: one of 'pascal_voc' (pre-trained on pascal voc),
            'cityscapes' (pre-trained on cityscape) or None (random initialization)
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: shape of input image. format HxWxC
            PASCAL VOC model was trained on (512,512,3) images. None is allowed as shape/width
        classes: number of desired classes. PASCAL VOC has 21 classes, Cityscapes has 19 classes.
            If number of classes not aligned with the weights used, last layer is initialized randomly
        backbone: backbone to use. one of {'xception','mobilenetv2'}
        activation: optional activation to add to the top of the network.
            One of 'softmax', 'sigmoid' or None
        OS: determines input_shape/feature_extractor_output ratio. One of {8,16}.
            Used only for xception backbone.
        alpha: controls the width of the MobileNetV2 network. This is known as the
            width multiplier in the MobileNetV2 paper.
                - If `alpha` < 1.0, proportionally decreases the number
                    of filters in each layer.
                - If `alpha` > 1.0, proportionally increases the number
                    of filters in each layer.
                - If `alpha` = 1, default number of filters from the paper
                    are used at each layer.
            Used only for mobilenetv2 backbone. Pretrained is only available for alpha=1.
    # Returns
        A Keras model instance.
    # Raises
        RuntimeError: If attempting to run this model with a
            backend that does not support separable convolutions.
        ValueError: in case of invalid argument for `weights` or `backbone`
    """

    if not (weights in {'pascal_voc', 'cityscapes', None}):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `pascal_voc`, or `cityscapes` '
                         '(pre-trained on PASCAL VOC)')

    if not (backbone in {'xception', 'mobilenetv2'}):
        raise ValueError('The `backbone` argument should be either '
                         '`xception`  or `mobilenetv2` ')

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        img_input = input_tensor

    if backbone == 'xception':
        if OS == 8:
            entry_block3_stride = 1
            middle_block_rate = 2  # ! Not mentioned in paper, but required
            exit_block_rates = (2, 4)
            atrous_rates = (12, 24, 36)
        else:
            entry_block3_stride = 2
            middle_block_rate = 1
            exit_block_rates = (1, 2)
            atrous_rates = (6, 12, 18)

        x = Conv2D(32, (3, 3), strides=(2, 2),
                   name='entry_flow_conv1_1', use_bias=False, padding='same')(img_input)
        x = BatchNormalization(name='entry_flow_conv1_1_BN')(x)
        x = Activation(tf.nn.relu)(x)

        x = _conv2d_same(x, 64, 'entry_flow_conv1_2', kernel_size=3, stride=1)
        x = BatchNormalization(name='entry_flow_conv1_2_BN')(x)
        x = Activation(tf.nn.relu)(x)

        x = _xception_block(x, [128, 128, 128], 'entry_flow_block1',
                            skip_connection_type='conv', stride=2,
                            depth_activation=False)
        x, skip1 = _xception_block(x, [256, 256, 256], 'entry_flow_block2',
                                   skip_connection_type='conv', stride=2,
                                   depth_activation=False, return_skip=True)

        x = _xception_block(x, [728, 728, 728], 'entry_flow_block3',
                            skip_connection_type='conv', stride=entry_block3_stride,
                            depth_activation=False)
        for i in range(16):
            x = _xception_block(x, [728, 728, 728], 'middle_flow_unit_{}'.format(i + 1),
                                skip_connection_type='sum', stride=1, rate=middle_block_rate,
                                depth_activation=False)

        x = _xception_block(x, [728, 1024, 1024], 'exit_flow_block1',
                            skip_connection_type='conv', stride=1, rate=exit_block_rates[0],
                            depth_activation=False)
        x = _xception_block(x, [1536, 1536, 2048], 'exit_flow_block2',
                            skip_connection_type='none', stride=1, rate=exit_block_rates[1],
                            depth_activation=True)

    else:
        OS = 8
        first_block_filters = _make_divisible(32 * alpha, 8)
        x = Conv2D(first_block_filters,
                   kernel_size=3,
                   strides=(2, 2), padding='same', use_bias=False,
                   name='Conv' if input_shape[2] == 3 else 'Conv_')(img_input)
        x = BatchNormalization(
            epsilon=1e-3, momentum=0.999, name='Conv_BN')(x)
        x = Activation(tf.nn.relu6, name='Conv_Relu6')(x)

        x = _inverted_res_block(x, filters=16, alpha=alpha, stride=1,
                                expansion=1, block_id=0, skip_connection=False)

        x = _inverted_res_block(x, filters=24, alpha=alpha, stride=2,
                                expansion=6, block_id=1, skip_connection=False)
        x = _inverted_res_block(x, filters=24, alpha=alpha, stride=1,
                                expansion=6, block_id=2, skip_connection=True)

        x = _inverted_res_block(x, filters=32, alpha=alpha, stride=2,
                                expansion=6, block_id=3, skip_connection=False)
        x = _inverted_res_block(x, filters=32, alpha=alpha, stride=1,
                                expansion=6, block_id=4, skip_connection=True)
        x = _inverted_res_block(x, filters=32, alpha=alpha, stride=1,
                                expansion=6, block_id=5, skip_connection=True)

        # stride in block 6 changed from 2 -> 1, so we need to use rate = 2
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1,  # 1!
                                expansion=6, block_id=6, skip_connection=False)
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=7, skip_connection=True)
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=8, skip_connection=True)
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=9, skip_connection=True)

        x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=10, skip_connection=False)
        x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=11, skip_connection=True)
        x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=12, skip_connection=True)

        x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1, rate=2,  # 1!
                                expansion=6, block_id=13, skip_connection=False)
        x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1, rate=4,
                                expansion=6, block_id=14, skip_connection=True)
        x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1, rate=4,
                                expansion=6, block_id=15, skip_connection=True)

        x = _inverted_res_block(x, filters=320, alpha=alpha, stride=1, rate=4,
                                expansion=6, block_id=16, skip_connection=False)

    # end of feature extractor

    # branching for Atrous Spatial Pyramid Pooling

    # Image Feature branch
    shape_before = tf.shape(x)
    b4 = GlobalAveragePooling2D()(x)
    b4_shape = tf.keras.backend.int_shape(b4)
    # from (b_size, channels)->(b_size, 1, 1, channels)
    b4 = Reshape((1, 1, b4_shape[1]))(b4)
    b4 = Conv2D(256, (1, 1), padding='same',
                use_bias=False, name='image_pooling')(b4)
    b4 = BatchNormalization(name='image_pooling_BN', epsilon=1e-5)(b4)
    b4 = Activation(tf.nn.relu)(b4)
    # upsample. have to use compat because of the option align_corners
    size_before = tf.keras.backend.int_shape(x)
    b4 = tf.keras.layers.experimental.preprocessing.Resizing(
            *size_before[1:3], interpolation="bilinear"
        )(b4)
    # simple 1x1
    b0 = Conv2D(256, (1, 1), padding='same', use_bias=False, name='aspp0')(x)
    b0 = BatchNormalization(name='aspp0_BN', epsilon=1e-5)(b0)
    b0 = Activation(tf.nn.relu, name='aspp0_activation')(b0)

    # there are only 2 branches in mobilenetV2. not sure why
    if backbone == 'xception':
        # rate = 6 (12)
        b1 = SepConv_BN(x, 256, 'aspp1',
                        rate=atrous_rates[0], depth_activation=True, epsilon=1e-5)
        # rate = 12 (24)
        b2 = SepConv_BN(x, 256, 'aspp2',
                        rate=atrous_rates[1], depth_activation=True, epsilon=1e-5)
        # rate = 18 (36)
        b3 = SepConv_BN(x, 256, 'aspp3',
                        rate=atrous_rates[2], depth_activation=True, epsilon=1e-5)

        # concatenate ASPP branches & project
        x = Concatenate()([b4, b0, b1, b2, b3])
    else:
        x = Concatenate()([b4, b0])

    x = Conv2D(256, (1, 1), padding='same',
               use_bias=False, name='concat_projection')(x)
    x = BatchNormalization(name='concat_projection_BN', epsilon=1e-5)(x)
    x = Activation(tf.nn.relu)(x)
    x = Dropout(0.1)(x)
    # DeepLab v.3+ decoder

    if backbone == 'xception':
        # Feature projection
        # x4 (x2) block
        skip_size = tf.keras.backend.int_shape(skip1)
        x = tf.keras.layers.experimental.preprocessing.Resizing(
                *skip_size[1:3], interpolation="bilinear"
            )(x)
        dec_skip1 = Conv2D(48, (1, 1), padding='same',
                           use_bias=False, name='feature_projection0')(skip1)
        dec_skip1 = BatchNormalization(
            name='feature_projection0_BN', epsilon=1e-5)(dec_skip1)
        dec_skip1 = Activation(tf.nn.relu)(dec_skip1)
        x = Concatenate()([x, dec_skip1])
        x = SepConv_BN(x, 256, 'decoder_conv0',
                       depth_activation=True, epsilon=1e-5)
        x = SepConv_BN(x, 256, 'decoder_conv1',
                       depth_activation=True, epsilon=1e-5)

    # you can use it with arbitary number of classes
    if (weights == 'pascal_voc' and classes == 21) or (weights == 'cityscapes' and classes == 19):
        last_layer_name = 'logits_semantic'
    else:
        last_layer_name = 'custom_logits_semantic'

    x = Conv2D(classes, (1, 1), padding='same', name=last_layer_name)(x)
    size_before3 = tf.keras.backend.int_shape(img_input)
    x = tf.keras.layers.experimental.preprocessing.Resizing(
            *size_before3[1:3], interpolation="bilinear"
        )(x)
    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    if activation in {'softmax', 'sigmoid'}:
        x = tf.keras.layers.Activation(activation)(x)

    model = Model(inputs, x, name='deeplabv3plus')

    # load weights

    if weights == 'pascal_voc':
        if backbone == 'xception':
            weights_path = get_file('deeplabv3_xception_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH_X,
                                    cache_subdir='models')
        else:
            weights_path = get_file('deeplabv3_mobilenetv2_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH_MOBILE,
                                    cache_subdir='models')
        model.load_weights(weights_path, by_name=True)
    elif weights == 'cityscapes':
        if backbone == 'xception':
            weights_path = get_file('deeplabv3_xception_tf_dim_ordering_tf_kernels_cityscapes.h5',
                                    WEIGHTS_PATH_X_CS,
                                    cache_subdir='models')
        else:
            weights_path = get_file('deeplabv3_mobilenetv2_tf_dim_ordering_tf_kernels_cityscapes.h5',
                                    WEIGHTS_PATH_MOBILE_CS,
                                    cache_subdir='models')
        model.load_weights(weights_path, by_name=True)
    return model

def preprocess_input(x):
    """Preprocesses a numpy array encoding a batch of images.
    # Arguments
        x: a 4D numpy array consists of RGB values within [0, 255].
    # Returns
        Input array scaled to [-1.,1.]
    """
    return preprocess_input(x, mode='tf')

In [ ]:
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.losses import binary_crossentropy
def tversky_coef(y_true, y_pred, alpha=0.7, const=K.epsilon()):
    '''
    Weighted Sørensen–Dice coefficient.
    
    Input
    ----------
        y_true, y_pred: predicted outputs and targets.
        const: a constant that smooths the loss gradient and reduces numerical instabilities.
        
    '''
    
    # flatten 2-d tensors
    y_true_pos = tf.reshape(y_true, [-1])
    y_pred_pos = tf.reshape(y_pred, [-1])
    
    # get true pos (TP), false neg (FN), false pos (FP).
    true_pos  = tf.reduce_sum(y_true_pos * y_pred_pos)
    false_neg = tf.reduce_sum(y_true_pos * (1-y_pred_pos))
    false_pos = tf.reduce_sum((1-y_true_pos) * y_pred_pos)
    
    # TP/(TP + a*FN + b*FP); a+b = 1
    coef_val = (true_pos + const)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + const)
    
    return coef_val

def tversky(y_true, y_pred, alpha=0.7, const=K.epsilon()):
    '''
    Tversky Loss.
    
    tversky(y_true, y_pred, alpha=0.5, const=K.epsilon())
    
    ----------
    Hashemi, S.R., Salehi, S.S.M., Erdogmus, D., Prabhu, S.P., Warfield, S.K. and Gholipour, A., 2018. 
    Tversky as a loss function for highly unbalanced image segmentation using 3d fully convolutional deep networks. 
    arXiv preprint arXiv:1803.11078.
    
    Input
    ----------
        alpha: tunable parameter within [0, 1]. Alpha handles imbalance classification cases.
        const: a constant that smooths the loss gradient and reduces numerical instabilities.
        
    '''
    # tf tensor casting
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    
    # <--- squeeze-out length-1 dimensions.
    y_pred = tf.squeeze(y_pred)
    y_true = tf.squeeze(y_true)
    
    loss_val = 1 - tversky_coef(y_true, y_pred, alpha=alpha, const=const)
    
    return loss_val

def focal_tversky(y_true, y_pred, alpha=0.7, gamma=4/3, const=K.epsilon()):
    
    '''
    Focal Tversky Loss (FTL)
    
    focal_tversky(y_true, y_pred, alpha=0.5, gamma=4/3)
    
    ----------
    Abraham, N. and Khan, N.M., 2019, April. A novel focal tversky loss function with improved 
    attention u-net for lesion segmentation. In 2019 IEEE 16th International Symposium on Biomedical Imaging 
    (ISBI 2019) (pp. 683-687). IEEE.
    
    ----------
    Input
        alpha: tunable parameter within [0, 1]. Alpha handles imbalance classification cases 
        gamma: tunable parameter within [1, 3].
        const: a constant that smooths the loss gradient and reduces numerical instabilities.
        
    '''
    # tf tensor casting
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    
    # <--- squeeze-out length-1 dimensions.
    y_pred = tf.squeeze(y_pred)
    y_true = tf.squeeze(y_true)
    
    # (Tversky loss)**(1/gamma) 
    loss_val = tf.math.pow((1-tversky_coef(y_true, y_pred, alpha=alpha, const=const)), 1/gamma)
    
    return loss_val


def dice_coef(y_true, y_pred, const=K.epsilon()):
    '''
    Sørensen–Dice coefficient for 2-d samples.
    
    Input
    ----------
        y_true, y_pred: predicted outputs and targets.
        const: a constant that smooths the loss gradient and reduces numerical instabilities.
        
    '''
    
    # flatten 2-d tensors
    y_true_pos = tf.reshape(y_true, [-1])
    y_pred_pos = tf.reshape(y_pred, [-1])
    
    # get true pos (TP), false neg (FN), false pos (FP).
    true_pos  = tf.reduce_sum(y_true_pos * y_pred_pos)
    false_neg = tf.reduce_sum(y_true_pos * (1-y_pred_pos))
    false_pos = tf.reduce_sum((1-y_true_pos) * y_pred_pos)
    
    # 2TP/(2TP+FP+FN) == 2TP/()
    coef_val = (2.0 * true_pos + const)/(2.0 * true_pos + false_pos + false_neg)
    
    return coef_val

def dice(y_true, y_pred, const=K.epsilon()):
    '''
    Sørensen–Dice Loss.
    
    dice(y_true, y_pred, const=K.epsilon())
    
    Input
    ----------
        const: a constant that smooths the loss gradient and reduces numerical instabilities.
        
    '''
    # tf tensor casting
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    
    # <--- squeeze-out length-1 dimensions.
    y_pred = tf.squeeze(y_pred)
    y_true = tf.squeeze(y_true)
    
    loss_val = 1 - dice_coef(y_true, y_pred, const=const)
    
    #BCE =  K.binary_crossentropy(y_true,y_pred)
    return loss_val # + BCE

def dsc(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dsc(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

def DiceBCELoss2(targets, inputs, smooth=1e-6):    
       
    #flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    BCE =  K.binary_crossentropy(targets, inputs)
    intersection = K.sum(inputs * targets)
    dice_loss = 1 - (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    Dice_BCE = BCE + dice_loss
    
    return Dice_BCE


ALPHA = 0.5 # < 0.5 penalises FP more, > 0.5 penalises FN more
CE_RATIO = 0.5 #weighted contribution of modified CE loss compared to Dice loss
smooth=1
def Combo_loss2(targets, inputs, eps=K.epsilon()):
    targets = K.flatten(targets)
    inputs = K.flatten(inputs)
    
    intersection = K.sum(targets * inputs)
    dice = (2. * intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    inputs = K.clip(inputs, eps, 1.0 - eps)
    out = - (ALPHA * ((targets * K.log(inputs)) + ((1 - ALPHA) * (1.0 - targets) * K.log(1.0 - inputs))))
    weighted_ce = K.mean(out, axis=-1)
    combo = (CE_RATIO * weighted_ce) - ((1 - CE_RATIO) * dice)
    
    return combo

opt = Adam(0.001)

model = Deeplabv3(input_shape=(512, 512, 3), classes=1, backbone='xception', activation='sigmoid')

model.compile(optimizer=opt, loss = focal_tversky, metrics=['accuracy', sm.metrics.IOUScore(threshold=0.5)]) 

history = model.fit(x_train,y_train,batch_size=4,epochs=100,verbose=1)



In [ ]:
#from keras_unet_collection.losses import  dice, iou_seg, focal_tversky, crps2d_tf

import tensorflow as tf 

from keras.losses import LossFunctionWrapper
from keras import backend as K
from keras.utils import losses_utils
from keras.utils import tf_utils
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import f1_score,accuracy_score

from keras.optimizers import SGD





def pixelwise_weighted_binary_crossentropy(y_true, y_pred):
    '''
    Pixel-wise weighted binary cross-entropy loss.
    The code is adapted from the Keras TF backend.
    (see their github)
    
    Parameters
    ----------
    y_true : Tensor
        Stack of groundtruth segmentation masks + weight maps.
    y_pred : Tensor
        Predicted segmentation masks.

    Returns
    -------
    Tensor
        Pixel-wise weight binary cross-entropy between inputs.

    '''
    
   
    [seg, weight] = tf.unstack(y_true, 2, axis=-1)

    seg = tf.expand_dims(seg, -1)
    weight = tf.expand_dims(weight, -1)
    

    epsilon = tf.convert_to_tensor(K.epsilon(), y_pred.dtype.base_dtype)
    y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
    y_pred = tf.math.log(y_pred / (1 - y_pred))

    zeros = array_ops.zeros_like(y_pred, dtype=y_pred.dtype)
    cond = (y_pred >= zeros)
    relu_logits = math_ops.select(cond, y_pred, zeros)
    neg_abs_logits = math_ops.select(cond, -y_pred, y_pred)
    entropy = math_ops.add(relu_logits - y_pred * seg, math_ops.log1p(math_ops.exp(neg_abs_logits)), name=None)
    
    # This is essentially the only part that is different from the Keras code:
    return K.mean(math_ops.multiply(weight, entropy), axis=-1)
    

def soft_dice_loss(y_true, y_pred, epsilon=1e-6):  # skip the batch and class axis for calculating Dice score a
    
    """Soft dice loss calculation for arbitrary batch size, number of classes, and number of spatial dimensions.
    Assumes the `channels_last` format.
  
    # Arguments
        y_true: b x X x Y( x Z...) x c One hot encoding of ground truth
        y_pred: b x X x Y( x Z...) x c Network output, must sum to 1 over c channel (such as after softmax) 
        epsilon: Used for numerical stability to avoid divide by zero errors
        # skip the batch and class axis for calculating Dice score
    axes = tuple(range(1, len(y_pred.shape)-1)) 
    numerator = 2. * np.sum(y_pred * y_true, axes)
    denominator = np.sum(np.square(y_pred) + np.square(y_true), axes)
    
    return 1 - np.mean(numerator / (denominator + epsilon)) # average over classes and batch
    """
    axes = tuple(range(1, len(y_pred.shape)-1)) 
    numerator = 2. * tf.reduce_sum(y_pred * y_true, axes) 
    denominator = tf.reduce_sum(tf.square(y_pred) + tf.square(y_true), axes) 
    result = 1 - tf.reduce_mean((numerator + epsilon) / (denominator + epsilon)) 
    return result # average over classes and batch

#Accuracy :  94.78267669677734 teeth binary

ce_w = 0.15
ce_d_w = 0.5
e = K.epsilon()
smooth = 1
'''
ce_w values smaller than 0.5 penalize false positives more while values larger than 0.5 penalize false negatives more
ce_d_w is level of contribution of the cross-entropy loss in the total loss.
'''

def Combo_loss(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    d = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    y_pred_f = K.clip(y_pred_f, e, 1.0 - e)
    out = - (ce_w * y_true_f * K.log(y_pred_f)) + ((1 - ce_w) * (1.0 - y_true_f) * K.log(1.0 - y_pred_f))
    weighted_ce = K.mean(out, axis=-1)
    combo = (ce_d_w * weighted_ce) - ((1 - ce_d_w) * d)
    return combo

ALPHA = 0.8
GAMMA = 2

def FocalLoss(targets, inputs, alpha=ALPHA, gamma=GAMMA):    
    
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    BCE = K.binary_crossentropy(targets, inputs)
    BCE_EXP = K.exp(-BCE)
    focal_loss = K.mean(alpha * K.pow((1-BCE_EXP), gamma) * BCE)
    
    return focal_loss

#Keras
def DiceBCELoss(targets, inputs, smooth=1e-6):    
       
    #flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    BCE =  K.binary_crossentropy(targets, inputs)
    intersection = K.sum(inputs * targets)
    dice_loss = 1 - (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    Dice_BCE = 0.8*BCE + 0.5*dice_loss
    
    return Dice_BCE


def tversky2(y_true, y_pred):
    smooth=1
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)
    

def dice_coef(y_true, y_pred, const=K.epsilon()):
    '''
    Sørensen–Dice coefficient for 2-d samples.
    
    Input
    ----------
        y_true, y_pred: predicted outputs and targets.
        const: a constant that smooths the loss gradient and reduces numerical instabilities.
        
    '''
    
    # flatten 2-d tensors
    y_true_pos = tf.reshape(y_true, [-1])
    y_pred_pos = tf.reshape(y_pred, [-1])
    
    # get true pos (TP), false neg (FN), false pos (FP).
    true_pos  = tf.reduce_sum(y_true_pos * y_pred_pos)
    false_neg = tf.reduce_sum(y_true_pos * (1-y_pred_pos))
    false_pos = tf.reduce_sum((1-y_true_pos) * y_pred_pos)
    
    # 2TP/(2TP+FP+FN) == 2TP/()
    coef_val = (2.0 * true_pos + const)/(2.0 * true_pos + false_pos + false_neg)
    
    return coef_val

def dice(y_true, y_pred, const=K.epsilon()):
    '''
    Sørensen–Dice Loss.
    
    dice(y_true, y_pred, const=K.epsilon())
    
    Input
    ----------
        const: a constant that smooths the loss gradient and reduces numerical instabilities.
        
    '''
    # tf tensor casting
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    
    # <--- squeeze-out length-1 dimensions.
    y_pred = tf.squeeze(y_pred)
    y_true = tf.squeeze(y_true)
    
    loss_val = 1 - dice_coef(y_true, y_pred, const=const)
    
    #BCE =  K.binary_crossentropy(y_true,y_pred)
    return loss_val # + BCE

def tversky_coef(y_true, y_pred, alpha=0.7, const=K.epsilon()):
    '''
    Weighted Sørensen–Dice coefficient.
    
    Input
    ----------
        y_true, y_pred: predicted outputs and targets.
        const: a constant that smooths the loss gradient and reduces numerical instabilities.
        
    '''
    
    # flatten 2-d tensors
    y_true_pos = tf.reshape(y_true, [-1])
    y_pred_pos = tf.reshape(y_pred, [-1])
    
    # get true pos (TP), false neg (FN), false pos (FP).
    true_pos  = tf.reduce_sum(y_true_pos * y_pred_pos)
    false_neg = tf.reduce_sum(y_true_pos * (1-y_pred_pos))
    false_pos = tf.reduce_sum((1-y_true_pos) * y_pred_pos)
    
    # TP/(TP + a*FN + b*FP); a+b = 1
    coef_val = (true_pos + const)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + const)
    
    return coef_val

def tversky(y_true, y_pred, alpha=0.7, const=K.epsilon()):
    '''
    Tversky Loss.
    
    tversky(y_true, y_pred, alpha=0.5, const=K.epsilon())
    
    ----------
    Hashemi, S.R., Salehi, S.S.M., Erdogmus, D., Prabhu, S.P., Warfield, S.K. and Gholipour, A., 2018. 
    Tversky as a loss function for highly unbalanced image segmentation using 3d fully convolutional deep networks. 
    arXiv preprint arXiv:1803.11078.
    
    Input
    ----------
        alpha: tunable parameter within [0, 1]. Alpha handles imbalance classification cases.
        const: a constant that smooths the loss gradient and reduces numerical instabilities.
        
    '''
    # tf tensor casting
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    
    # <--- squeeze-out length-1 dimensions.
    y_pred = tf.squeeze(y_pred)
    y_true = tf.squeeze(y_true)
    
    loss_val = 1 - tversky_coef(y_true, y_pred, alpha=alpha, const=const)
    
    return loss_val

def focal_tversky(y_true, y_pred, alpha=0.7, gamma=4/3, const=K.epsilon()):
    
    '''
    Focal Tversky Loss (FTL)
    
    focal_tversky(y_true, y_pred, alpha=0.5, gamma=4/3)
    
    ----------
    Abraham, N. and Khan, N.M., 2019, April. A novel focal tversky loss function with improved 
    attention u-net for lesion segmentation. In 2019 IEEE 16th International Symposium on Biomedical Imaging 
    (ISBI 2019) (pp. 683-687). IEEE.
    
    ----------
    Input
        alpha: tunable parameter within [0, 1]. Alpha handles imbalance classification cases 
        gamma: tunable parameter within [1, 3].
        const: a constant that smooths the loss gradient and reduces numerical instabilities.
        
    '''
    # tf tensor casting
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    
    # <--- squeeze-out length-1 dimensions.
    y_pred = tf.squeeze(y_pred)
    y_true = tf.squeeze(y_true)
    
    # (Tversky loss)**(1/gamma) 
    loss_val = tf.math.pow((1-tversky_coef(y_true, y_pred, alpha=alpha, const=const)), 1/gamma)
    
    return loss_val

def suggest_filter_size(image1_batch,image2_batch,power_factors,filter_size):
      shape1= image1_batch.shape[1:-1]  
      shape2= image2_batch.shape[1:-1] 
      if not(shape1[-3:-1][0]/(2**(len(power_factors)-1)) and shape2[-3:-1][0]/(2**(len(power_factors)-1)) >= filter_size):
            H = tf.math.reduce_min((shape1,shape2))
            suggested_filter_size = int(H/(2**(len(power_factors)-1)))
      else:
            suggested_filter_size = filter_size
      return suggested_filter_size

def ms_ssim(y_true, y_pred):
    """
    Multiscale structural similarity (MS-SSIM) loss.
    
    ms_ssim(y_true, y_pred, **tf_ssim_kw)
    
    ----------
    Wang, Z., Simoncelli, E.P. and Bovik, A.C., 2003, November. Multiscale structural similarity for image quality assessment. 
    In The Thrity-Seventh Asilomar Conference on Signals, Systems & Computers, 2003 (Vol. 2, pp. 1398-1402). Ieee.
    
    ----------
    Input
        kwargs: keywords of `tf.image.ssim_multiscale`
                https://www.tensorflow.org/api_docs/python/tf/image/ssim_multiscale
                
        *Issues of `tf.image.ssim_multiscale`refers to:
                https://stackoverflow.com/questions/57127626/error-in-calculation-of-inbuilt-ms-ssim-function-in-tensorflow
    
    """
    
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    
    y_pred = tf.squeeze(y_pred)
    y_true = tf.squeeze(y_true)
    
    
    
    tf_ms_ssim = tf.image.ssim_multiscale(y_true, y_pred, max_val=1.0,filter_size=1)
        
    return 1 - tf_ms_ssim

def DiceBCELoss2(targets, inputs, smooth=1e-6):    
       
    #flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    BCE =  K.binary_crossentropy(targets, inputs)
    intersection = K.sum(inputs * targets)
    dice_loss = 1 - (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    Dice_BCE = BCE + dice_loss
    
    return Dice_BCE


BACKBONE = 'resnet50'
LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)



model = sm.Unet(BACKBONE, encoder_weights='imagenet', classes=1, activation='sigmoid', input_shape=(512, 512, 3))

model.compile(optimizer=optim, loss =  focal_tversky, metrics=['accuracy',sm.metrics.IOUScore(threshold=0.5)]) 



# callbacks=[early_stopping_monitor],
history = model.fit(x_train,y_train,batch_size=4,epochs=100,verbose=1)





In [ ]:
import gc
gc.collect()

In [ ]:
predict_img=model.predict(x_test)


In [ ]:
print(history.history.keys())
print(history.history['f1-score'])
plt.plot(history.history['f1-score'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
from keras.models import load_model

def DiceBCELoss(targets, inputs, smooth=1e-6):    
       
    #flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    BCE =  K.binary_crossentropy(targets, inputs)
    intersection = K.sum(inputs * targets)
    dice_loss = 1 - (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    Dice_BCE = BCE + dice_loss
    
    return Dice_BCE

model= load_model("Croppedv3loveu.h5", compile=False)
model.compile(optimizer ='adam', loss = DiceBCELoss, metrics = ['accuracy'])

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score,jaccard_score, roc_auc_score
def dice(pred, true, k = 1):
    intersection = np.sum(pred[true==k]) * 2.0
    dice = intersection / (np.sum(pred) + np.sum(true))
    return dice




def plot_side_by_side(original_image, predicted_image, rad, label, pp, dice_score):

    print('Plotting ' + label)
    a, (orig_ax, pred_ax, rad_ax) = plt.subplots(1, 3, figsize=(60,15))
    orig_ax.set_axis_off()
    orig_ax.imshow(original_image, aspect="auto")
    orig_ax.set_title("Original", fontsize = 160)
    pred_ax.set_axis_off()
    pred_ax.imshow(predicted_image, aspect="auto")
    pred_ax.set_title("Predicted", fontsize = 160)
    rad_ax.set_axis_off()
    rad_ax.imshow(rad, aspect="auto")
    rad_ax.set_title("Radiography", fontsize = 160)
    
    txt = "Dice: {:.6f}".format(dice_score)
    plt.text(0.95,0.5,txt, transform=a.transFigure, size=100)
    pp.savefig(a, pad_inches=2, bbox_inches='tight')
 
    

  
    plt.show()
    
    
pp = PdfPages('unet102.pdf')


true_list_new=[]
pred_list_new=[]

dices=[]

for i in range(len(x_test)):
   
    
    
    predict_img[i] = np.where(predict_img[i] > 0.25, 1, 0)
    predict_img1=(predict_img[i]>0.001)*1

    y_test1=(y_test[i]>0.001)*1
    dice_score = dice(predict_img1, y_test1, k = 1)
    print ("Dice Similarity: {}".format(dice_score))
    plot_side_by_side(y_test[i], predict_img[i], x_test[i], str(i), pp, dice_score)
    true_list_new.append(y_test1)
    pred_list_new.append(predict_img1)
    dices.append(dice_score)
    
   
    
    
    


true_list_new=np.array(true_list_new)
pred_list_new=np.array(pred_list_new)

true_list_new=true_list_new.flatten()
pred_list_new=pred_list_new.flatten()

print ("Accuracy : ", accuracy_score(true_list_new,pred_list_new)*100) 
print ("Dice : ", dice(true_list_new, pred_list_new))

print ("IOU : ", jaccard_score(true_list_new, pred_list_new))
print ("F1 : ",  f1_score(true_list_new, pred_list_new))

print('Recall: %.3f' % recall_score(true_list_new, pred_list_new))
print('Precision: %.3f' % precision_score(true_list_new, pred_list_new))
print('AUC: %.3f' % roc_auc_score(true_list_new, pred_list_new))

txt1 = "Accuracy : " + str(accuracy_score(true_list_new,pred_list_new)*100)
txt2 = "Dice : " + str(dice(true_list_new, pred_list_new))
txt3 = "IOU : "+ str(jaccard_score(true_list_new, pred_list_new))
txt4 = "F1 : " +  str(f1_score(true_list_new, pred_list_new))
txt5 = "Recall: {:.3f}".format(recall_score(true_list_new, pred_list_new))
txt6 = "Precision: {:.3f}".format(precision_score(true_list_new, pred_list_new))
txt7 = "AUC: {:.3f}".format(roc_auc_score(true_list_new, pred_list_new))

fig = plt.figure(figsize=(110.69,80.27))
plt.text(0.05,0.95,txt1, transform=fig.transFigure, size=124)
plt.text(0.05,0.85,txt2, transform=fig.transFigure, size=124)
plt.text(0.05,0.75,txt3, transform=fig.transFigure, size=124)
plt.text(0.05,0.65,txt4, transform=fig.transFigure, size=124)
plt.text(0.05,0.55,txt5, transform=fig.transFigure, size=124)
plt.text(0.05,0.45,txt6, transform=fig.transFigure, size=124)
plt.text(0.05,0.35,txt7, transform=fig.transFigure, size=124)
pp.savefig()
pp.close()
    

In [ ]:
model.save("11deeplabfocal_tversky0486.h5")

In [ ]:
!pip install imutils

In [ ]:
def crop_image_v2(img, t):
    mask = img!=0
    mask = mask.any(2)
    mask0,mask1 = mask.any(0),mask.any(1)
    colstart, colend = mask0.argmax(), len(mask0)-mask0[::-1].argmax()+1
    rowstart, rowend = mask1.argmax(), len(mask1)-mask1[::-1].argmax()+1
    return t[rowstart:rowend, colstart:colend]



In [ ]:
from keras.models import load_model

def dice(y_true, y_pred, const=K.epsilon()):
    '''
    Sørensen–Dice Loss.
    
    dice(y_true, y_pred, const=K.epsilon())
    
    Input
    ----------
        const: a constant that smooths the loss gradient and reduces numerical instabilities.
        
    '''
    # tf tensor casting
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    
    # <--- squeeze-out length-1 dimensions.
    y_pred = tf.squeeze(y_pred)
    y_true = tf.squeeze(y_true)
    
    loss_val = 1 - dice_coef(y_true, y_pred, const=const)
    
    #BCE =  K.binary_crossentropy(y_true,y_pred)
    return loss_val # + BCE

model= load_model("detection_teeth", compile=False)
#model.compile(optimizer ='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.compile(optimizer ='adam', loss = dice, metrics = ['accuracy'])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def crop_img(img, scale=1.0):
    center_x, center_y = img.shape[1] / 2, img.shape[0] / 2
    width_scaled, height_scaled = img.shape[1] * scale, img.shape[0] * scale
    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
    img_cropped = img[int(top_y):int(bottom_y), int(left_x):int(right_x)]
    return img_cropped

img=X[22]
img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
img_cropped = crop_img(img, 0.70)

new_x=[]
for img in X:
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    img_cropped = crop_img(img, 0.70)
    width = int(512)
    height = int(512)
    dim = (width, height)

    # resize image
    img_cropped = cv2.resize(img_cropped, dim, interpolation = cv2.INTER_AREA)
    new_x.append(img_cropped)

plt.imshow(new_x[1])
    

In [ ]:
import imageio
import cv2
from matplotlib.backends.backend_pdf import PdfPages
def crop_image_v2(img, t):
    mask = img!=0
    
    mask = mask.any(2)
    mask0,mask1 = mask.any(0),mask.any(1)
    colstart, colend = mask0.argmax(), len(mask0)-mask0[::-1].argmax()+1
    rowstart, rowend = mask1.argmax(), len(mask1)-mask1[::-1].argmax()+1
    return t[rowstart:rowend, colstart:colend]





for i in range(len(Y)):
    teeth=X[i]
    teeth2=Y[i]
    img=cv2.imread(files[i])


    new_img=crop_image_v2(img, teeth)
    new_img2=crop_image_v2(img, teeth2)
    resize_shape=(512,512)
    new_img = cv2.resize(new_img, (512,512))
    new_img2 = cv2.resize(new_img2, (512,512))
    
    imageio.imwrite(os.path.join('croppedX101',str(i) + '.jpg'),new_img)
    imageio.imwrite(os.path.join('croppedY101',str(i) + '.jpg'),new_img2)




In [ ]:
for i in range(len(predict_img)):
    predicted=predict_img[i]
    imageio.imwrite(os.path.join('tempo3',str(i) + '.jpg'),predicted)

In [ ]:
import imageio
#imageio.imwrite('outfile.jpg',predict_img[1])

In [ ]:
subdirec='tempo3/'
files = os.listdir(subdirec)
files.sort(key= lambda filename: int(filename.split('.')[0]))
files = [os.path.join(subdirec, f) for f in files]
print(files)

In [ ]:
teeth=x_test[1]
img=predict_img[1]
def crop_image_v2(img, t):
    mask = img!=0
    mask = mask.any(2)
    mask0,mask1 = mask.any(0),mask.any(1)
    colstart, colend = mask0.argmax(), len(mask0)-mask0[::-1].argmax()+1
    rowstart, rowend = mask1.argmax(), len(mask1)-mask1[::-1].argmax()+1
    return t[rowstart:rowend, colstart:colend]

new_img=crop_image_v2(img, teeth)

In [ ]:
subdirec='/kaggle/input/croppedcavities/croppedX/'
files = os.listdir(subdirec)
files.sort(key= lambda filename: int(filename.split('.')[0]))
files = [os.path.join(subdirec, f) for f in files]
print(files)

In [ ]:
from keras import backend as K
def tversky_coef(y_true, y_pred, alpha=0.7, const=K.epsilon()):
    '''
    Weighted Sørensen–Dice coefficient.

    Input
    ----------
        y_true, y_pred: predicted outputs and targets.
        const: a constant that smooths the loss gradient and reduces numerical instabilities.

    '''

    # flatten 2-d tensors
    y_true_pos = tf.reshape(y_true, [-1])
    y_pred_pos = tf.reshape(y_pred, [-1])

    # get true pos (TP), false neg (FN), false pos (FP).
    true_pos = tf.reduce_sum(y_true_pos * y_pred_pos)
    false_neg = tf.reduce_sum(y_true_pos * (1 - y_pred_pos))
    false_pos = tf.reduce_sum((1 - y_true_pos) * y_pred_pos)

    # TP/(TP + a*FN + b*FP); a+b = 1
    const = 1
    coef_val = (true_pos + const) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + const)

    return coef_val


def tversky(y_true, y_pred, alpha=0.7, const=K.epsilon()):
    '''
    Tversky Loss.

    tversky(y_true, y_pred, alpha=0.5, const=K.epsilon())

    ----------
    Hashemi, S.R., Salehi, S.S.M., Erdogmus, D., Prabhu, S.P., Warfield, S.K. and Gholipour, A., 2018.
    Tversky as a loss function for highly unbalanced image segmentation using 3d fully convolutional deep networks.
    arXiv preprint arXiv:1803.11078.

    Input
    ----------
        alpha: tunable parameter within [0, 1]. Alpha handles imbalance classification cases.
        const: a constant that smooths the loss gradient and reduces numerical instabilities.

    '''
    # tf tensor casting
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)

    # <--- squeeze-out length-1 dimensions.
    y_pred = tf.squeeze(y_pred)
    y_true = tf.squeeze(y_true)

    loss_val = 1 - tversky_coef(y_true, y_pred, alpha=alpha, const=const)

    return loss_val
from keras.models import load_model
model1= load_model("/kaggle/input/uploads/11unet_tversky054.h5", compile=False)
model1.compile(optimizer ='adam', loss = tversky, metrics = ['accuracy'])

In [ ]:
def focal_tversky(y_true, y_pred, alpha=0.7, gamma=4/3, const=K.epsilon()):
    
    '''
    Focal Tversky Loss (FTL)
    
    focal_tversky(y_true, y_pred, alpha=0.5, gamma=4/3)
    
    ----------
    Abraham, N. and Khan, N.M., 2019, April. A novel focal tversky loss function with improved 
    attention u-net for lesion segmentation. In 2019 IEEE 16th International Symposium on Biomedical Imaging 
    (ISBI 2019) (pp. 683-687). IEEE.
    
    ----------
    Input
        alpha: tunable parameter within [0, 1]. Alpha handles imbalance classification cases 
        gamma: tunable parameter within [1, 3].
        const: a constant that smooths the loss gradient and reduces numerical instabilities.
        
    '''
    # tf tensor casting
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    
    # <--- squeeze-out length-1 dimensions.
    y_pred = tf.squeeze(y_pred)
    y_true = tf.squeeze(y_true)
    
    # (Tversky loss)**(1/gamma) 
    loss_val = tf.math.pow((1-tversky_coef(y_true, y_pred, alpha=alpha, const=const)), 1/gamma)
    
    
    return loss_val

model2= load_model("/kaggle/input/uploads/11fpnfocaltversky05091.h5", compile=False)
model2.compile(optimizer ='adam', loss = focal_tversky, metrics = ['accuracy'])

In [ ]:

model3= load_model("/kaggle/input/uploads/11deeplabfocal_tversky0486.h5", compile=False)
model3.compile(optimizer ='adam', loss = focal_tversky, metrics = ['accuracy'])

In [ ]:
predict_img1=model1.predict(x_test)
predict_img2=model2.predict(x_test)
predict_img3=model3.predict(x_test)

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score,jaccard_score, roc_auc_score
def dice(pred, true, k = 1):
    intersection = np.sum(pred[true==k]) * 2.0
    dice = intersection / (np.sum(pred) + np.sum(true))
    return dice




def plot_side_by_side(original_image, predicted_image1, predicted_image2, predicted_image3, predicted_image4, rad, label, pp, dice_score):

    print('Plotting ' + label)
    a, (rad_ax, orig_ax, pred_ax1) = plt.subplots(1, 3, figsize=(60,15)) # figsize=(45,5)) #pred_ax2, pred_ax3, pred_ax4
    rad_ax.set_axis_off()
    rad_ax.imshow(rad, aspect="auto")
    rad_ax.set_title("Radiography", fontsize = 60)
    orig_ax.set_axis_off()
    orig_ax.imshow(original_image, aspect="auto")
    orig_ax.set_title("Original", fontsize = 60)
    pred_ax1.set_axis_off()
    pred_ax1.imshow(predicted_image4, aspect="auto")
    pred_ax1.set_title("Combined", fontsize = 60)
    """
    pred_ax2.set_axis_off()
    pred_ax2.imshow(predicted_image1, aspect="auto")
    pred_ax2.set_title("Unet", fontsize = 60)
    pred_ax3.set_axis_off()
    pred_ax3.imshow(predicted_image2, aspect="auto")
    pred_ax3.set_title("Fpn", fontsize = 60)
    pred_ax4.set_axis_off()
    pred_ax4.imshow(predicted_image3, aspect="auto")
    pred_ax4.set_title("Deeplab", fontsize = 60)
    """
    
    
    txt = "Dice: {:.6f}".format(dice_score)
    plt.text(0.95,0.5,txt, transform=a.transFigure, size=100)
    pp.savefig(a, pad_inches=2, bbox_inches='tight')
 
    

  
    plt.show()
    
    
pp = PdfPages('11combined_dice068justcombined2.pdf')


true_list_new=[]
pred_list_new=[]

dices=[]
predict_img4=[]
#predict_img=y_pred
#print(len(y_pred))
for i in range(len(x_test)):
   
    
    combined = predict_img1[i]  + predict_img2[i] + predict_img3[i]
    predict_img1[i] = np.where(predict_img1[i] > 0.15, 1, 0)
    predict_img2[i] = np.where(predict_img2[i] > 0.15, 1, 0)
    predict_img3[i] = np.where(predict_img3[i] > 0.45, 1, 0)
    combined = np.where(combined >= 1.05, 1, 0)
    
    predict_img11=(combined>0.001)*1

    y_test1=(y_test[i]>0.001)*1
        #score=f1_score(y_test1[i].flatten(), predict_img1[i].flatten(), average='micro')
        #print(score)
    dice_score = dice(predict_img11, y_test1, k = 1) #255 in my case, can be 1 
    print ("Dice Similarity: {}".format(dice_score))
    plot_side_by_side(y_test[i], predict_img1[i], predict_img2[i], predict_img3[i], combined, x_test[i], str(i), pp, dice_score)
    true_list_new.append(y_test1)
    pred_list_new.append(predict_img11)
    dices.append(dice_score)
    
   
    
    
    


true_list_new=np.array(true_list_new)
pred_list_new=np.array(pred_list_new)

true_list_new=true_list_new.flatten()
pred_list_new=pred_list_new.flatten()

print ("Accuracy : ", accuracy_score(true_list_new,pred_list_new)*100) 
print ("Dice : ", dice(true_list_new,pred_list_new)) 

print ("IOU : ", jaccard_score(true_list_new, pred_list_new))
print ("F1 : ",  f1_score(true_list_new, pred_list_new))

print('Recall: %.3f' % recall_score(true_list_new, pred_list_new))
print('Precision: %.3f' % precision_score(true_list_new, pred_list_new))
print('AUC: %.3f' % roc_auc_score(true_list_new, pred_list_new))

txt1 = "Accuracy : " + str(accuracy_score(true_list_new,pred_list_new)*100)
txt2 = "Dice : " + str(dice(true_list_new,pred_list_new))
txt3 = "IOU : "+ str(jaccard_score(true_list_new, pred_list_new))
txt4 = "F1 : " +  str(f1_score(true_list_new, pred_list_new))
txt5 = "Recall: {:.3f}".format(recall_score(true_list_new, pred_list_new))
txt6 = "Precision: {:.3f}".format(precision_score(true_list_new, pred_list_new))
txt7 = "AUC: {:.3f}".format(roc_auc_score(true_list_new, pred_list_new))

fig = plt.figure(figsize=(110.69,80.27))
plt.text(0.05,0.95,txt1, transform=fig.transFigure, size=124)
plt.text(0.05,0.85,txt2, transform=fig.transFigure, size=124)
plt.text(0.05,0.75,txt3, transform=fig.transFigure, size=124)
plt.text(0.05,0.65,txt4, transform=fig.transFigure, size=124)
plt.text(0.05,0.55,txt5, transform=fig.transFigure, size=124)
plt.text(0.05,0.45,txt6, transform=fig.transFigure, size=124)
plt.text(0.05,0.35,txt7, transform=fig.transFigure, size=124)
pp.savefig()
pp.close()